# Zillow

In [18]:
import re
import datetime as dt
import json

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [19]:
#config.py

OW_ZIP_URL="http://api.openweathermap.org/geo/1.0/zip?zip={zip},US&appid={api_key}"

ZILLOW_HOMES_URL="https://www.zillow.com/{city}-{state}/{property_type}/{page}_p"

HOME_HEADER_CLASS=("div",{"class":"StyledPropertyCardDataWrapper-c11n-8-85-1__sc-1omp4c3-0 jVBMsP property-card-data"})
HOME_PAGE_CLASS=("span",{"class":"Text-c11n-8-85-1__sc-aiai24-0 bEkett"})
HOME_PRICE_CLASS=("div",{"class":"StyledPropertyCardDataArea-c11n-8-85-1__sc-yipmu-0 bqsBln"})
HOME_SPACE_CLASS=("ul",{"class":"StyledPropertyCardHomeDetailsList-c11n-8-85-1__sc-1xvdaej-0 dmDolk"})
HOME_ADDRESS_CLASS=("a",{"class":"StyledPropertyCardDataArea-c11n-8-85-1__sc-yipmu-0 gdfTyO property-card-link"})

LIST_ITEM_CLASS="li"
LIST_ITEM_IDENTIFIER="abbr"

ERROR_404_CONSTANT='zillow-error-page'
BOT_CAUGHT_CONSTANT='robots'

In [20]:
city_states=[('Houston','TX')]

In [21]:
#main.py

l=[]
h={}

for city, state in city_states:  
    while True:
        num_var=np.random.randint(100,1000)
        num_var3=np.random.randint(10,100)
        num_var2=num_var3%10
        num_var4=np.random.randint(1000,10000)
        num_var5=np.random.randint(100,1000)

        _header={"User-Agent": f"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/{num_var5}.36 (KHTML, like Gecko) Chrome/51.{num_var2}.2704.{num_var} Safari/537.{num_var3} OPR/38.0.{num_var4}.41"}
        _page_req=requests.get(ZILLOW_HOMES_URL.format(city=city.lower(),
                                                        state=state.lower(),
                                                        property_type='homes',
                                                        page='2'),headers=_header)

        _page_text = _page_req.text

        _page_soup=BeautifulSoup(_page_text,'html.parser')

        try:
            if _page_soup.find('div')['id']==ERROR_404_CONSTANT:
                break
        except:
            pass
        
        if _page_soup.find('meta')['name']==BOT_CAUGHT_CONSTANT:
            continue
        else:
            if _page_soup.find(HOME_PAGE_CLASS[0],HOME_PAGE_CLASS[1]) == None:
                _page_max=1
            else:
                _target_text=_page_soup.find(HOME_PAGE_CLASS[0],HOME_PAGE_CLASS[1]).text

                _page_max=int(re.findall('(\d+)',_target_text)[1])
            break


    for p_idx in range(2,(_page_max+2)):
        try:
            if _page_soup.find('div')['id']==ERROR_404_CONSTANT:
                break
        except:
                pass
            
        while True:
            num_var=np.random.randint(100,1000)
            num_var3=np.random.randint(10,100)
            num_var2=num_var3%10
            num_var4=np.random.randint(1000,10000)
            num_var5=np.random.randint(100,1000)

            header={"User-Agent": f"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/{num_var5}.36 (KHTML, like Gecko) Chrome/51.{num_var2}.2704.{num_var} Safari/537.{num_var3} OPR/38.0.{num_var4}.41"}
            req = requests.get(ZILLOW_HOMES_URL.format(city=city.lower(),
                                                        state=state.lower(),
                                                        property_type='homes',
                                                        page=p_idx),headers=header)
            
            r_text=req.text


            soup = BeautifulSoup(r_text,'html.parser')


            if soup.find('meta')['name']==BOT_CAUGHT_CONSTANT:
                continue

            else:

                homes = soup.find_all(HOME_HEADER_CLASS[0],HOME_HEADER_CLASS[1])

                for home in homes:

                    p_list=home.find(HOME_PRICE_CLASS[0],HOME_PRICE_CLASS[1]).text
                    try:
                        h['price']=int("".join(re.findall('(\d+)',p_list)))
                    except:
                        h['price']=None


                    s_list=home.find(HOME_SPACE_CLASS[0],HOME_SPACE_CLASS[1])
                    try:
                        s_list_list = s_list.find_all('li')
                        for li_idx in range(len(s_list_list)):
                            li_identifier = s_list_list[li_idx].find('abbr')  
                            try:
                                if re.findall("(bd)", li_identifier.text)[0] == 'bd': 
                                    h['bd'] = int("".join(re.findall('(\d+)', s_list_list[li_idx].text)))
                            except:
                                pass
                        
                            try:
                                if re.findall("(ba)", li_identifier.text)[0] == 'ba':  
                                    h['ba'] = int("".join(re.findall('(\d+)', s_list_list[li_idx].text)))
                            except:
                                pass
                            
                            try:
                                if re.findall("(sqft)", li_identifier.text)[0] == 'sqft':
                                    h['sqft'] = int("".join(re.findall('(\d+)', s_list_list[li_idx].text)))
                            except:
                                pass
                    except:
                        pass


                    a_list=home.find(HOME_ADDRESS_CLASS[0],HOME_ADDRESS_CLASS[1]).text.split(",")
                    try:
                        h['address']=a_list[0].strip()
                    except:
                        h['address']=None
                    try:
                        h['city']=a_list[1].strip()
                    except:
                        h['city']=None
                    try:
                        h['state']=a_list[2].strip().split()[0]
                    except:
                        h['state']=None
                    try:
                        h['zip']=a_list[2].strip().split()[1]
                    except:
                        h['zip']=None


                    header={"content-type": "application/json"}
                    req=requests.get(OW_ZIP_URL.format(zip=h['zip'],api_key="c603cf27fe72100c8d3aaf7d7a409c86"),headers=header)

                    json_loads=json.loads(req.text)

                    try:
                        h['lat']=float(json_loads['lat'])
                        h['lon']=float(json_loads['lon'])
                    except:
                        h['lat']=None
                        h['lon']=None




                    l.append(h)
                    h={}
                break


In [22]:
df=pd.DataFrame(l)

In [23]:
df

,price,bd,ba,sqft,address,city,state,zip,lat,lon
0,289900,3.0,3.0,2154.0,834 Hillock Bluff Cir,Houston,TX,77073,30.0198,-95.4087
1,290000,3.0,3.0,2140.0,6307 Wilshire Rdg,Houston,TX,77040,29.8744,-95.5278
2,274400,4.0,2.0,2134.0,1502 Chestnut Ridge Rd,Humble,TX,77339,30.0569,-95.1835
3,200000,3.0,2.0,1410.0,10534 Kirkgreen Dr,Houston,TX,77089,29.5940,-95.2218
4,489900,5.0,5.0,3300.0,202 Ogilvie St,Houston,TX,77017,29.6863,-95.2555
...,...,...,...,...,...,...,...,...,...,...
175,279999,4.0,3.0,2544.0,3903 Stillview Dr,Houston,TX,77068,30.0069,-95.4897
176,150000,2.0,2.0,1008.0,4572 Dunnam Rd,Kingwood,TX,77345,30.0336,-95.2610
177,250000,5.0,3.0,3810.0,722 Vista Grove Cir,Houston,TX,77073,30.0198,-95.4087
178,199999,3.0,2.0,1715.0,3715 Vineyard Dr,Houston,TX,77082,29.7223,-95.6285


INCORPORATE MEMOIZATION AND THREADING